==============================================================================
04 - TRAIN MODELS FOR ADDITIONAL HORIZONS (3 and 5 bars ahead)
==============================================================================

Train XGBoost, CNN-LSTM, and TCN-Attention for horizons 3 and 5.
Horizon 1 models trained in notebooks 02, 03, 03b.

Each training cell is independent after data loading.


# 04 - Train Additional Horizons

Train all 3 model types for horizons 3 and 5.


In [1]:
# ==============================================================================
# IMPORTS
# ==============================================================================

import sys
from pathlib import Path

# Works both as script (__file__ exists) and in Jupyter (use cwd)
import os
try:
    project_root = Path(__file__).resolve().parent.parent
except NameError:
    cwd = Path(os.getcwd())
    project_root = cwd.parent if cwd.name == 'notebooks' else cwd
sys.path.insert(0, str(project_root))

import numpy as np
import json
import warnings
warnings.filterwarnings('ignore')

from src.data.loader import load_and_merge_data, split_data_by_time
from src.labeling.oracle import create_oracle_labels
from src.features.builder import prepare_features
from src.features.indicators import get_indicator_columns
from src.models.xgb import XGBBaseline
from src.models.cnn_lstm import CNNLSTMModel
from src.models.tcn_attention import TCNAttentionModel

print("✅ Imports loaded")


✅ Imports loaded


In [2]:
# ==============================================================================
# CONFIGURATION
# ==============================================================================

SIGMA = 4
THRESHOLD = 0.0002
TRAIN_END = "2025-06-30"
TEST_START = "2025-07-01"
MODEL_DIR = 'models_artifacts'

# Load best TCN params if available (from grid search)
try:
    with open(f'{MODEL_DIR}/tcn_attention_best_params.json', 'r') as f:
        TCN_BEST_PARAMS = json.load(f)
    print(f"📋 Loaded TCN best params from grid search")
except:
    # Default params if grid search not run
    TCN_BEST_PARAMS = {
        'tcn_filters': 64,
        'num_tcn_blocks': 3,
        'lookback': 32,
        'dropout': 0.2,
        'use_class_weights': False
    }
    print(f"📋 Using default TCN params (run 03b for tuned params)")

all_results = {}

print(f"   Oracle: sigma={SIGMA}, threshold={THRESHOLD}")


📋 Loaded TCN best params from grid search
   Oracle: sigma=4, threshold=0.0002


In [3]:
# ==============================================================================
# LOAD DATA
# ==============================================================================

print("=" * 60)
print("📥 LOADING DATA")
print("=" * 60)

df = load_and_merge_data(end_date='2025-12-31')
df = create_oracle_labels(df, sigma=SIGMA, threshold=THRESHOLD)
print(f"✅ Loaded {len(df):,} rows")


📥 LOADING DATA

📥 LOADING DATA

✅ Loading pre-merged dataset: /mnt/c/_PYTH/projects/capstone_project/data/processed/all_merged.parquet
⏰ Filtered until: 2025-12-31

📊 Data Summary:
   Rows: 70,081
   Columns: 21
   Date range: 2024-01-01 00:00:00+00:00 to 2025-12-31 00:00:00+00:00
✅ Loaded 70,081 rows


---
## HORIZON 3 (45 minutes ahead)
---


In [4]:
# ==============================================================================
# PREPARE DATA FOR HORIZON 3
# ==============================================================================

HORIZON = 3
print(f"\n{'='*60}")
print(f"🎯 PREPARING DATA FOR HORIZON={HORIZON}")
print("="*60)

all_results[HORIZON] = {}

df_features_h3, _ = prepare_features(df.copy(), horizon=HORIZON)
train_df_h3, val_df_h3, test_df_h3 = split_data_by_time(
    df_features_h3, train_end=TRAIN_END, test_start=TEST_START, val_ratio=0.1
)

feature_cols = get_indicator_columns(
    df_features_h3, exclude_cols=['time', 'target', 'smoothed_close', 'smooth_slope']
)
feature_cols = [c for c in feature_cols if c in train_df_h3.columns]

X_train_h3 = np.nan_to_num(train_df_h3[feature_cols].values, nan=0.0, posinf=0.0, neginf=0.0)
y_train_h3 = train_df_h3['target'].values.astype(int)
X_val_h3 = np.nan_to_num(val_df_h3[feature_cols].values, nan=0.0, posinf=0.0, neginf=0.0)
y_val_h3 = val_df_h3['target'].values.astype(int)
X_test_h3 = np.nan_to_num(test_df_h3[feature_cols].values, nan=0.0, posinf=0.0, neginf=0.0)
y_test_h3 = test_df_h3['target'].values.astype(int)

print(f"   Train: {len(train_df_h3):,}, Val: {len(val_df_h3):,}, Test: {len(test_df_h3):,}")



🎯 PREPARING DATA FOR HORIZON=3

🔧 PREPARING FEATURES
Groups to add: ['momentum', 'overlap', 'trend', 'volatility', 'volume', 'statistics', 'candle']

📊 Adding indicators...
  -> Processing group: momentum
     Added 14 features: STOCHRSIk_14_14_3_3, RSI_14, TSIs_13_25_13, TSI_13_25_13, CCI_14_0.015...
  -> Processing group: overlap
[!] VWAP requires an ordered DatetimeIndex.
     Added 7 features: EMA_20, SMA_200, EMA_100, EMA_50, TEMA_9...
  -> Processing group: trend
     Added 17 features: TRIX_30_9, AROOND_14, CKSPl_10_3_20, VTXP_14, TRIXs_30_9...
  -> Processing group: volatility
       BBL_20_2.0_2.0  BBM_20_2.0_2.0  BBU_20_2.0_2.0  BBB_20_2.0_2.0  \
0                 NaN             NaN             NaN             NaN   
1                 NaN             NaN             NaN             NaN   
2                 NaN             NaN             NaN             NaN   
3                 NaN             NaN             NaN             NaN   
4                 NaN             NaN     

In [5]:
# ==============================================================================
# TRAIN XGBOOST FOR HORIZON 3
# ==============================================================================

print(f"\n{'─'*60}")
print(f"🌲 TRAINING XGBOOST (H={HORIZON})")
print("─"*60)

xgb_h3 = XGBBaseline(n_classes=3, device='cuda', random_state=42)
xgb_h3.fit(X_train_h3, y_train_h3, X_val_h3, y_val_h3, feature_names=feature_cols)
xgb_h3.tune(X_train_h3, y_train_h3, n_iter=15, cv_splits=3, scoring='f1_weighted')
xgb_metrics_h3 = xgb_h3.evaluate(X_test_h3, y_test_h3)

print(f"✅ Acc={xgb_metrics_h3['accuracy']:.4f}, F1={xgb_metrics_h3['f1_weighted']:.4f}")
Path(MODEL_DIR).mkdir(exist_ok=True)
xgb_h3.save(MODEL_DIR, name='xgb_baseline_h3')
all_results[3]['XGBoost'] = xgb_metrics_h3



────────────────────────────────────────────────────────────
🌲 TRAINING XGBOOST (H=3)
────────────────────────────────────────────────────────────

🔧 Tuning hyperparameters with TimeSeriesSplit...
Fitting 3 folds for each of 15 candidates, totalling 45 fits

✅ Best CV score: 0.4945
Best parameters: {'subsample': 1.0, 'reg_lambda': 0.1, 'reg_alpha': 1, 'n_estimators': 100, 'min_child_weight': 3, 'max_depth': 3, 'learning_rate': 0.01, 'gamma': 0, 'colsample_bytree': 0.6}
✅ Acc=0.5363, F1=0.5210
✅ Model saved to models_artifacts


In [6]:
# ==============================================================================
# TRAIN CNN-LSTM FOR HORIZON 3
# ==============================================================================

print(f"\n{'─'*60}")
print(f"🧠 TRAINING CNN-LSTM (H={HORIZON})")
print("─"*60)

cnn_h3 = CNNLSTMModel(
    n_classes=3, lookback=32, conv_filters=64, lstm_units=64,
    dropout=0.3, learning_rate=0.0007, device='cuda'
)
cnn_h3.fit(X_train_h3, y_train_h3, X_val_h3, y_val_h3, epochs=30, batch_size=128, patience=5)
cnn_metrics_h3 = cnn_h3.evaluate(X_test_h3, y_test_h3)

print(f"✅ Acc={cnn_metrics_h3['accuracy']:.4f}, F1={cnn_metrics_h3['f1_weighted']:.4f}")
cnn_h3.save(MODEL_DIR, name='cnn_lstm_h3')
all_results[3]['CNN-LSTM'] = cnn_metrics_h3



────────────────────────────────────────────────────────────
🧠 TRAINING CNN-LSTM (H=3)
────────────────────────────────────────────────────────────
🚀 GPU detected: /physical_device:GPU:0

📊 Preparing data...
📊 Creating sequences (lookback=32)...
   Train sequences: (46915, 32, 67)
   Val sequences: (5184, 32, 67)

🏗️ Building model...


I0000 00:00:1769586246.588629  128827 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21720 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "cnn_lstm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 32, 67)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 32, 64)         │        12,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_conv1 (BatchNormalization)   │ (None, 32, 64)         │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu_conv1 (Activation)         │ (None, 32, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_conv1 (Dropout)         │ (None, 32, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 32, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_conv2 (BatchNormalization)   │ (None, 32, 128)        │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu_conv2 (Activation)         │ (None, 32, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_conv2 (Dropout)         │ (None, 32, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32, 64)         │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_lstm1 (BatchNormalization)   │ (None, 32, 64)         │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_lstm1 (Dropout)         │ (None, 32, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 191,107 (746.51 KB)

 Trainable params: 190,595 (744.51 KB)

 Non-trainable params: 512 (2.00 KB)


⚖️ Class weights: {np.int32(0): np.float64(1.0922149275969641), np.int32(1): np.float64(0.934580370126895), np.int32(2): np.float64(0.9857749201546478)}

🏋️ Training for up to 30 epochs (patience=5)...
Epoch 1/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 12s 17ms/step - accuracy: 0.4587 - loss: 1.0391 - val_accuracy: 0.4296 - val_loss: 1.0806 - learning_rate: 7.0000e-04
Epoch 2/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.5007 - loss: 0.9975 - val_accuracy: 0.4485 - val_loss: 1.0875 - learning_rate: 7.0000e-04
Epoch 3/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.5335 - loss: 0.9586 - val_accuracy: 0.4460 - val_loss: 1.1123 - learning_rate: 7.0000e-04
Epoch 4/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.5646 - loss: 0.9123 - val_accuracy: 0.4201 - val_loss: 1.1925 - learning_rate: 7.0000e-04
Epoch 5/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.6016 - loss: 0.8565 - val_accuracy: 0.4130 - val_loss: 1.2222 - learning_rate: 7.0000e-04
Epoch 6/30


In [7]:
# ==============================================================================
# TRAIN TCN-ATTENTION FOR HORIZON 3
# ==============================================================================

print(f"\n{'─'*60}")
print(f"⚡ TRAINING TCN-ATTENTION (H={HORIZON})")
print("─"*60)

tcn_h3 = TCNAttentionModel(
    n_classes=3,
    lookback=TCN_BEST_PARAMS.get('lookback', 32),
    tcn_filters=TCN_BEST_PARAMS.get('tcn_filters', 64),
    num_tcn_blocks=TCN_BEST_PARAMS.get('num_tcn_blocks', 3),
    dropout=TCN_BEST_PARAMS.get('dropout', 0.2),
    learning_rate=0.0007,
    device='cuda'
)
tcn_h3.fit(
    X_train_h3, y_train_h3, X_val_h3, y_val_h3,
    epochs=30, batch_size=128, patience=5,
    use_class_weights=TCN_BEST_PARAMS.get('use_class_weights', False)
)
tcn_metrics_h3 = tcn_h3.evaluate(X_test_h3, y_test_h3)

print(f"✅ Acc={tcn_metrics_h3['accuracy']:.4f}, F1={tcn_metrics_h3['f1_weighted']:.4f}")
tcn_h3.save(MODEL_DIR, name='tcn_attention_h3')
all_results[3]['TCN-Attention'] = tcn_metrics_h3



────────────────────────────────────────────────────────────
⚡ TRAINING TCN-ATTENTION (H=3)
────────────────────────────────────────────────────────────
🚀 GPU detected: /physical_device:GPU:0

📊 Preparing data...
📊 Creating sequences (lookback=16)...
   Train sequences: (46931, 16, 67)
   Val sequences: (5200, 16, 67)

🏗️ Building TCN-Attention model...


Model: "tcn_attention"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 16, 67)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ tcn_block_1 (TCNBlock)          │ (None, 16, 128)        │        35,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ tcn_block_2 (TCNBlock)          │ (None, 16, 128)        │        49,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ tcn_block_3 (TCNBlock)          │ (None, 16, 128)        │        49,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention                       │ (None, 16, 128)        │        66,304 │
│ (MultiHeadSelfAttention)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gap (GlobalAveragePooling1D)    │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 205,187 (801.51 KB)

 Trainable params: 204,419 (798.51 KB)

 Non-trainable params: 768 (3.00 KB)


🏋️ Training for up to 30 epochs (patience=5)...
Epoch 1/30
 14/367 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.3604 - loss: 1.2670

I0000 00:00:1769586306.054264  129144 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


367/367 ━━━━━━━━━━━━━━━━━━━━ 20s 32ms/step - accuracy: 0.4363 - loss: 1.0933 - val_accuracy: 0.4285 - val_loss: 1.0954 - learning_rate: 7.0000e-04
Epoch 2/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4700 - loss: 1.0607 - val_accuracy: 0.4454 - val_loss: 1.0920 - learning_rate: 7.0000e-04
Epoch 3/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4904 - loss: 1.0368 - val_accuracy: 0.4065 - val_loss: 1.1279 - learning_rate: 7.0000e-04
Epoch 4/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5099 - loss: 1.0113 - val_accuracy: 0.4075 - val_loss: 1.1405 - learning_rate: 7.0000e-04
Epoch 5/30
356/367 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5228 - loss: 0.9934
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0002099999925121665.
367/367 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5310 - loss: 0.9836 - val_accuracy: 0.4379 - val_loss: 1.1380 - learning_rate: 7.0000e-04
Epoch 6/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5722 - loss:

---
## HORIZON 5 (75 minutes ahead)
---


In [8]:
# ==============================================================================
# PREPARE DATA FOR HORIZON 5
# ==============================================================================

HORIZON = 5
print(f"\n{'='*60}")
print(f"🎯 PREPARING DATA FOR HORIZON={HORIZON}")
print("="*60)

all_results[HORIZON] = {}

df_features_h5, _ = prepare_features(df.copy(), horizon=HORIZON)
train_df_h5, val_df_h5, test_df_h5 = split_data_by_time(
    df_features_h5, train_end=TRAIN_END, test_start=TEST_START, val_ratio=0.1
)

feature_cols = get_indicator_columns(
    df_features_h5, exclude_cols=['time', 'target', 'smoothed_close', 'smooth_slope']
)
feature_cols = [c for c in feature_cols if c in train_df_h5.columns]

X_train_h5 = np.nan_to_num(train_df_h5[feature_cols].values, nan=0.0, posinf=0.0, neginf=0.0)
y_train_h5 = train_df_h5['target'].values.astype(int)
X_val_h5 = np.nan_to_num(val_df_h5[feature_cols].values, nan=0.0, posinf=0.0, neginf=0.0)
y_val_h5 = val_df_h5['target'].values.astype(int)
X_test_h5 = np.nan_to_num(test_df_h5[feature_cols].values, nan=0.0, posinf=0.0, neginf=0.0)
y_test_h5 = test_df_h5['target'].values.astype(int)

print(f"   Train: {len(train_df_h5):,}, Val: {len(val_df_h5):,}, Test: {len(test_df_h5):,}")



🎯 PREPARING DATA FOR HORIZON=5

🔧 PREPARING FEATURES
Groups to add: ['momentum', 'overlap', 'trend', 'volatility', 'volume', 'statistics', 'candle']

📊 Adding indicators...
  -> Processing group: momentum
     Added 14 features: STOCHRSIk_14_14_3_3, RSI_14, TSIs_13_25_13, TSI_13_25_13, CCI_14_0.015...
  -> Processing group: overlap
[!] VWAP requires an ordered DatetimeIndex.
     Added 7 features: EMA_20, SMA_200, EMA_100, EMA_50, TEMA_9...
  -> Processing group: trend
     Added 17 features: TRIX_30_9, AROOND_14, CKSPl_10_3_20, VTXP_14, TRIXs_30_9...
  -> Processing group: volatility
       BBL_20_2.0_2.0  BBM_20_2.0_2.0  BBU_20_2.0_2.0  BBB_20_2.0_2.0  \
0                 NaN             NaN             NaN             NaN   
1                 NaN             NaN             NaN             NaN   
2                 NaN             NaN             NaN             NaN   
3                 NaN             NaN             NaN             NaN   
4                 NaN             NaN     

In [9]:
# ==============================================================================
# TRAIN XGBOOST FOR HORIZON 5
# ==============================================================================

print(f"\n{'─'*60}")
print(f"🌲 TRAINING XGBOOST (H={HORIZON})")
print("─"*60)

xgb_h5 = XGBBaseline(n_classes=3, device='cuda', random_state=42)
xgb_h5.fit(X_train_h5, y_train_h5, X_val_h5, y_val_h5, feature_names=feature_cols)
xgb_h5.tune(X_train_h5, y_train_h5, n_iter=15, cv_splits=3, scoring='f1_weighted')
xgb_metrics_h5 = xgb_h5.evaluate(X_test_h5, y_test_h5)

print(f"✅ Acc={xgb_metrics_h5['accuracy']:.4f}, F1={xgb_metrics_h5['f1_weighted']:.4f}")
xgb_h5.save(MODEL_DIR, name='xgb_baseline_h5')
all_results[5]['XGBoost'] = xgb_metrics_h5



────────────────────────────────────────────────────────────
🌲 TRAINING XGBOOST (H=5)
────────────────────────────────────────────────────────────

🔧 Tuning hyperparameters with TimeSeriesSplit...
Fitting 3 folds for each of 15 candidates, totalling 45 fits

✅ Best CV score: 0.4301
Best parameters: {'subsample': 1.0, 'reg_lambda': 0.1, 'reg_alpha': 1, 'n_estimators': 100, 'min_child_weight': 3, 'max_depth': 3, 'learning_rate': 0.01, 'gamma': 0, 'colsample_bytree': 0.6}
✅ Acc=0.4944, F1=0.4688
✅ Model saved to models_artifacts


In [10]:
# ==============================================================================
# TRAIN CNN-LSTM FOR HORIZON 5
# ==============================================================================

print(f"\n{'─'*60}")
print(f"🧠 TRAINING CNN-LSTM (H={HORIZON})")
print("─"*60)

cnn_h5 = CNNLSTMModel(
    n_classes=3, lookback=32, conv_filters=64, lstm_units=64,
    dropout=0.3, learning_rate=0.0007, device='cuda'
)
cnn_h5.fit(X_train_h5, y_train_h5, X_val_h5, y_val_h5, epochs=30, batch_size=128, patience=5)
cnn_metrics_h5 = cnn_h5.evaluate(X_test_h5, y_test_h5)

print(f"✅ Acc={cnn_metrics_h5['accuracy']:.4f}, F1={cnn_metrics_h5['f1_weighted']:.4f}")
cnn_h5.save(MODEL_DIR, name='cnn_lstm_h5')
all_results[5]['CNN-LSTM'] = cnn_metrics_h5



────────────────────────────────────────────────────────────
🧠 TRAINING CNN-LSTM (H=5)
────────────────────────────────────────────────────────────
🚀 GPU detected: /physical_device:GPU:0

📊 Preparing data...
📊 Creating sequences (lookback=32)...
   Train sequences: (46915, 32, 67)
   Val sequences: (5184, 32, 67)

🏗️ Building model...


Model: "cnn_lstm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 32, 67)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 32, 64)         │        12,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_conv1 (BatchNormalization)   │ (None, 32, 64)         │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu_conv1 (Activation)         │ (None, 32, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_conv1 (Dropout)         │ (None, 32, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 32, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_conv2 (BatchNormalization)   │ (None, 32, 128)        │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu_conv2 (Activation)         │ (None, 32, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_conv2 (Dropout)         │ (None, 32, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32, 64)         │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_lstm1 (BatchNormalization)   │ (None, 32, 64)         │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_lstm1 (Dropout)         │ (None, 32, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 191,107 (746.51 KB)

 Trainable params: 190,595 (744.51 KB)

 Non-trainable params: 512 (2.00 KB)


⚖️ Class weights: {np.int32(0): np.float64(1.0922149275969641), np.int32(1): np.float64(0.9346920885382424), np.int32(2): np.float64(0.9856506575906551)}

🏋️ Training for up to 30 epochs (patience=5)...
Epoch 1/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - accuracy: 0.4399 - loss: 1.0552 - val_accuracy: 0.3574 - val_loss: 1.1193 - learning_rate: 7.0000e-04
Epoch 2/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.4891 - loss: 1.0110 - val_accuracy: 0.3395 - val_loss: 1.1834 - learning_rate: 7.0000e-04
Epoch 3/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.5330 - loss: 0.9561 - val_accuracy: 0.3708 - val_loss: 1.2282 - learning_rate: 7.0000e-04
Epoch 4/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.5764 - loss: 0.8950 - val_accuracy: 0.3530 - val_loss: 1.2989 - learning_rate: 7.0000e-04
Epoch 5/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.6204 - loss: 0.8254 - val_accuracy: 0.3540 - val_loss: 1.3981 - learning_rate: 7.0000e-04
Epoch 6/30

In [11]:
# ==============================================================================
# TRAIN TCN-ATTENTION FOR HORIZON 5
# ==============================================================================

print(f"\n{'─'*60}")
print(f"⚡ TRAINING TCN-ATTENTION (H={HORIZON})")
print("─"*60)

tcn_h5 = TCNAttentionModel(
    n_classes=3,
    lookback=TCN_BEST_PARAMS.get('lookback', 32),
    tcn_filters=TCN_BEST_PARAMS.get('tcn_filters', 64),
    num_tcn_blocks=TCN_BEST_PARAMS.get('num_tcn_blocks', 3),
    dropout=TCN_BEST_PARAMS.get('dropout', 0.2),
    learning_rate=0.0007,
    device='cuda'
)
tcn_h5.fit(
    X_train_h5, y_train_h5, X_val_h5, y_val_h5,
    epochs=30, batch_size=128, patience=5,
    use_class_weights=TCN_BEST_PARAMS.get('use_class_weights', False)
)
tcn_metrics_h5 = tcn_h5.evaluate(X_test_h5, y_test_h5)

print(f"✅ Acc={tcn_metrics_h5['accuracy']:.4f}, F1={tcn_metrics_h5['f1_weighted']:.4f}")
tcn_h5.save(MODEL_DIR, name='tcn_attention_h5')
all_results[5]['TCN-Attention'] = tcn_metrics_h5



────────────────────────────────────────────────────────────
⚡ TRAINING TCN-ATTENTION (H=5)
────────────────────────────────────────────────────────────
🚀 GPU detected: /physical_device:GPU:0

📊 Preparing data...
📊 Creating sequences (lookback=16)...
   Train sequences: (46931, 16, 67)
   Val sequences: (5200, 16, 67)

🏗️ Building TCN-Attention model...


Model: "tcn_attention"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 16, 67)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ tcn_block_1 (TCNBlock)          │ (None, 16, 128)        │        35,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ tcn_block_2 (TCNBlock)          │ (None, 16, 128)        │        49,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ tcn_block_3 (TCNBlock)          │ (None, 16, 128)        │        49,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention                       │ (None, 16, 128)        │        66,304 │
│ (MultiHeadSelfAttention)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gap (GlobalAveragePooling1D)    │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 205,187 (801.51 KB)

 Trainable params: 204,419 (798.51 KB)

 Non-trainable params: 768 (3.00 KB)


🏋️ Training for up to 30 epochs (patience=5)...
Epoch 1/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 14s 20ms/step - accuracy: 0.4186 - loss: 1.1042 - val_accuracy: 0.4298 - val_loss: 1.1062 - learning_rate: 7.0000e-04
Epoch 2/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4410 - loss: 1.0791 - val_accuracy: 0.4163 - val_loss: 1.1124 - learning_rate: 7.0000e-04
Epoch 3/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4581 - loss: 1.0590 - val_accuracy: 0.4256 - val_loss: 1.1405 - learning_rate: 7.0000e-04
Epoch 4/30
362/367 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4705 - loss: 1.0461
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0002099999925121665.
367/367 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4761 - loss: 1.0400 - val_accuracy: 0.4206 - val_loss: 1.1810 - learning_rate: 7.0000e-04
Epoch 5/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5238 - loss: 0.9869 - val_accuracy: 0.3985 - val_loss: 1.2714 - learning_rate: 2.1000e-04
Epoch 6/30
367/367

---
## FINAL SUMMARY
---


In [12]:
# ==============================================================================
# SUMMARY
# ==============================================================================

print("\n" + "=" * 60)
print("📋 TRAINING COMPLETE - SUMMARY")
print("=" * 60)

print(f"\n{'Horizon':<10} {'Model':<15} {'Accuracy':<12} {'F1 Weighted':<12}")
print("─" * 50)

for h in [3, 5]:
    if h in all_results:
        for model in ['XGBoost', 'CNN-LSTM', 'TCN-Attention']:
            if model in all_results[h]:
                m = all_results[h][model]
                print(f"{h:<10} {model:<15} {m['accuracy']:.4f}       {m['f1_weighted']:.4f}")

print(f"""
{'─' * 50}

📁 Models saved to: {MODEL_DIR}/
   • xgb_baseline_h3/h5
   • cnn_lstm_h3/h5
   • tcn_attention_h3/h5

🔜 NEXT: Run 05_comparison for full comparison.
""")



📋 TRAINING COMPLETE - SUMMARY

Horizon    Model           Accuracy     F1 Weighted 
──────────────────────────────────────────────────
3          XGBoost         0.5363       0.5210
3          CNN-LSTM        0.4423       0.4461
3          TCN-Attention   0.4504       0.4401
5          XGBoost         0.4944       0.4688
5          CNN-LSTM        0.4025       0.4105
5          TCN-Attention   0.4566       0.4116

──────────────────────────────────────────────────

📁 Models saved to: models_artifacts/
   • xgb_baseline_h3/h5
   • cnn_lstm_h3/h5
   • tcn_attention_h3/h5

🔜 NEXT: Run 05_comparison for full comparison.

